# Motivation

This is an example notebook for using the expressiveness of notebooks for EDA of a dataset before moving onto the robustness of pure python scripts for modelling and serving

In [1]:
from sklearn.datasets import load_iris

In [2]:
iris = load_iris()

In [3]:
print(iris["DESCR"])

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [4]:
# This is a dataset for classifying species of iris' based of measurements of their petals and sepals.

In [5]:
list(iris.target_names)

['setosa', 'versicolor', 'virginica']

In [6]:
# We can use this database as a simple example project to show the end to end ML workflow and how to structure it for 
# use in a production system and for working on in a team.

In [7]:
import pandas as pd
import numpy as np

df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])

In [8]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [9]:
# We have the species encoded as a number which is useful for modelling but it's going to see the actual species
species = {
    0: "setosa",
    1: "versicolor",
    2: "virginica"
}
df["species"] = df["target"].apply(lambda x: species[x])
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target,species
0,5.1,3.5,1.4,0.2,0.0,setosa
1,4.9,3.0,1.4,0.2,0.0,setosa
2,4.7,3.2,1.3,0.2,0.0,setosa
3,4.6,3.1,1.5,0.2,0.0,setosa
4,5.0,3.6,1.4,0.2,0.0,setosa


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
sepal length (cm)    150 non-null float64
sepal width (cm)     150 non-null float64
petal length (cm)    150 non-null float64
petal width (cm)     150 non-null float64
target               150 non-null float64
species              150 non-null object
dtypes: float64(5), object(1)
memory usage: 7.1+ KB


In [11]:
# No nulls to impute, (real life is never this easy)
# what are some summary statistics of the variables to get an understading

In [12]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [13]:
# It always makes sense to scale these quantitative variables 
# but as we can see here they are defintely on different length scales 
# so scaling will be important to understand feature importance

In [14]:
# Are the target variables equally distributed? If they are not we can get a biased classifier when optimising for accuracy
# as it will preferentially classify to the majority class

df["target"].value_counts()

2.0    50
1.0    50
0.0    50
Name: target, dtype: int64

In [15]:
# As the target is equally distributed we will not have this problem

# What about correlation between any of the variables?
# Are there redundant variables which can be removed?
# A simpler model is always preferrable but certain models assume independent variables
# so you should not be using these models with correlated variables.

df.corr()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
sepal length (cm),1.000000,-0.117570,0.871754,0.817941,0.782561
sepal width (cm),-0.117570,1.000000,-0.428440,-0.366126,-0.426658
petal length (cm),0.871754,-0.428440,1.000000,0.962865,0.949035
petal width (cm),0.817941,-0.366126,0.962865,1.000000,0.956547
target,0.782561,-0.426658,0.949035,0.956547,1.000000


In [16]:
# Petal length and width are highly correlated 
# (as would be expected if you think of the analogy of height and weight in a person)

# What is very interesting though is how correlated petal width and length are with the target 
# these are going to be powerful features in predicting the species of iris

In [17]:
# We know the high level correlation but a more visual representation of this is always useful.
# A pair plot will show us a scatter graph of each feature rather than just a single value for the correlation.

import seaborn as sns

sns.pairplot(df, hue='species')

/home/jwgwalton/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval
/home/jwgwalton/anaconda3/lib/python3.7/site-packages/statsmodels/nonparametric/kde.py:488: RuntimeWarning: invalid value encountered in true_divide
  binned = fast_linbin(X, a, b, gridsize) / (delta * nobs)
/home/jwgwalton/anaconda3/lib/python3.7/site-packages/statsmodels/nonparametric/kdetools.py:34: RuntimeWarning: invalid value encountered in double_scalars
  FAC1 = 2*(np.pi*bw/RANGE)**2
/home/jwgwalton/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:83: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, *

In [18]:
# The pair plot lets us see the separability of individual features for the different species.

# Interestingly it looks like setosa is perfectly separable on petal length or petal width

# we could have a heuristic tsuch as 
# if petal_width < 1.0cm:
#    species = "setosa"

# For versicolor and virginica we will need the complexity of an ML model  to differentiate

# Modelling

Though i've stated that modelling should be done outside of a notebook (see README.md at top level for discussion)

In a notebook we can do model selection and write the final modelling code in a separate script. This allows for better reproducability when someone else is trying to work on your code.


### What we've learnt from the EDA
- There are no missing values to deal with through either imputation or ignoring the rows
- We need to scale the features so they are treated equally in the model (some models are robust to this such as ensembles of trees but it will never affect us badly)
- There is no class imbalance to deal with (the species are equally distributed)
- Some features are correlated (which may affect models which assume feature independence such as Naive Bayes)


### Aims for model selection

- Dumb baseline
- Compare different algorithms

In [19]:
y = df["target"].values

columns = [
    'sepal length (cm)', 
    'sepal width (cm)', 
    'petal length (cm)',
    'petal width (cm)', 
]
X = df[columns].values

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [21]:
# Always try and use a pipeline object for composability
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

# Scale data based of the minimum and maximum observed value to the interval [0, 1]
scaler = MinMaxScaler()
# A good baseline model, can be done for multi class classification and doesn't require independent features so we can throw all the features at it
lr = LogisticRegression(multi_class='auto') # specific algorithm for multi class problems

pipeline = Pipeline([
    ("scaler", scaler),
    ("lr", lr),
])

fitted_pipeline = pipeline.fit(X_train, y_train)

In [22]:
predictions = fitted_pipeline.predict(X_test)

In [23]:
# A great package for getting a confusion matrix and other statistics
from pycm import ConfusionMatrix

confusion_matrix = ConfusionMatrix(actual_vector=y_test, predict_vector=predictions)
print(confusion_matrix)

Predict   0.0       1.0       2.0       
Actual
0.0       7         0         0         

1.0       0         11        1         

2.0       0         0         11        





Overall Statistics : 

95% CI                                                            (0.90243,1.0309)
ACC Macro                                                         0.97778
ARI                                                               0.88499
AUNP                                                              0.97368
AUNU                                                              0.97734
Bennett S                                                         0.95
CBA                                                               0.94444
CSI                                                               0.94444
Chi-Squared                                                       55.20833
Chi-Squared DF                                                    4
Conditional Entropy                                       

In [24]:
# Unsurprisingly the setosa is predicted perfectly as we previously noted that it's perfectly separable. 
# This is incredibly good performance as we've only got 1 misclassification but lets try some different algorithms to compare

In [25]:
scaler = MinMaxScaler()
lr = LogisticRegression(multi_class='auto')

pipeline = Pipeline([
    ("scaler", scaler),
    ("lr", lr),
])

lr_pipeline = pipeline.fit(X_train, y_train)

from sklearn.naive_bayes import MultinomialNB

scaler = MinMaxScaler()
nb = MultinomialNB() # give it a go even though the correlated features may be a problem

pipeline = Pipeline([
    ("scaler", scaler),
    ("nb", nb),
])

nb_pipeline = pipeline.fit(X_train, y_train)

from sklearn.ensemble import RandomForestClassifier 
# This may overfit as it can model complex feature interractions
# It has lots of hyperparameters which can vary the model complexity hugely

scaler = MinMaxScaler()
rf = RandomForestClassifier()

pipeline = Pipeline([
    ("scaler", scaler),
    ("rf", rf),
])
rf_pipeline = pipeline.fit(X_train, y_train)

# Model assesment

Normally i would overlay plots of ROC curves for binary classification problems for easy comparison and we can visually see the the difference, it is possible to do this for multi class but i'm going to compare the confusion matrices and make a choice of the model to use.

Note: It is very important to do lots of different assesments of the model validity and dig into the misclassifications to identify commonalities which will often lead to specific feature engineering strategies but we're concnetrating on the end to end process of delivering an ML project rather than tuning hyperparams and the other small gains.

In [34]:
from sklearn.metrics import roc_auc_score

lr_predictions = lr_pipeline.predict(X_test)
nb_predictions = nb_pipeline.predict(X_test)
rf_predictions = rf_pipeline.predict(X_test)

In [39]:
lr_cm = ConfusionMatrix(actual_vector=y_test, predict_vector=lr_predictions)
nb_cm = ConfusionMatrix(actual_vector=y_test, predict_vector=nb_predictions)
rf_cm = ConfusionMatrix(actual_vector=y_test, predict_vector=rf_predictions)
print("Logistic Regression:\n")
lr_cm.print_matrix()
print("Naive Bayes AUC:\n")
nb_cm.print_matrix()
print("Random Forest AUC:\n")
rf_cm.print_matrix()

Logistic Regression:

Predict   0.0       1.0       2.0       
Actual
0.0       7         0         0         

1.0       0         11        1         

2.0       0         0         11        


Naive Bayes AUC:

Predict   0.0       1.0       2.0       
Actual
0.0       7         0         0         

1.0       0         0         12        

2.0       0         1         10        


Random Forest AUC:

Predict   0.0       1.0       2.0       
Actual
0.0       7         0         0         

1.0       0         11        1         

2.0       0         0         11        




- Unsurprisingly the Naive bayes struggled, if we got rid of the highly correlated features it may do better
- It' hard to beat 1 misclassification and the Randdom forest is just as good as the Logistic regression.

It's important to note that these results are actually so good that it's normally a warning sign that you've done something wrong and you should double check the data for any data leakage (is there a feature which is actually a proxy for the target that you wouldn't really have at run time).

# Conclusion

I'm going to use a logistic regression algorithm to create our final model.

I've skipped over a lot of things that occur in a real project 

    - Feature Selection (Removing redundant or uninformative features)
    - Feature Engineering (Combining or transforming raw input into features which allow the model to 
    - Model tuning (optimising the hyperparameters of the individual algorith)
    
All of these stages are iterative and you often find yourself going back to look at the data again based off insights gathered from looking at the misclassifications in the model and these then inform the creation of new features.

The most powerful thing you can do is improve the features and get sufficient data. model tuning will get you some small gains but the most important thing is to improve your input. Always be suspicious of the results and try and fully understand what the algorithm is actually doing. I always belabour the point when helping new data scientists that just looking at accuracy isn't helpful. For a spam filter with 1% of the emails 

From now on we move onto focusing on reproducibility and robustness. Creating a script which is testably correct for training our final model for use in a production system.